In [1]:
from scripts.software_frs import frs_traceability2
FRS = frs_traceability2
from officelib.xllib import *
from officelib.const import xlconst as xlc
import sys
import os
import re

from scripts.tools.issuetracker import IssuetrackerAPI
from datetime import datetime

import jpnotebooks.Software.SDLC_traceability_tools.issuetracker_item_extracter as iie

In [79]:
_cache = None
_age = None
def _download_issues():
    api = IssuetrackerAPI('issue.pbsbiotech.com', 'nstarkweather', 'kookychemist')
    return api.download_issues("pbssoftware", status_id="*")

def get_issues():
    global _cache, _age
    if not _cache or (datetime.now() - _age).total_seconds() > (8*3600):  # 8 hr
        _cache = _download_issues()
        _age = datetime.now()
    return _cache

In [80]:
def filter_relevant_issues(issues, rel_cb):
    relevant = {}
    for v in issues.values():
        if rel_cb(v):
                relevant[v.id] = v
    return relevant

def load_frs_from_issuetracker(rel_cb):
    issues = get_issues()
    relevant = filter_relevant_issues(issues, rel_cb)
    
    reqs = iie.IssuetrackerParser(["URS", "FRS", "SDS"]).parse_all(relevant.values())
    all_frs = {r.tag: r for r in reqs}
    return all_frs, relevant

In [137]:
def paste_data(ws, data):
    cells = ws.Cells
    cr = cells.Range
    
    di = 2
    hi = 3
    
    header_start = cr("A1")
    
    frs_start = cr("A2")
    frs_end = header_start.Offset(len(data), 1)

    id_start = frs_start.Offset(1, 3)
    id_end = frs_end.Offset(1, 3)
    

    paste_start = header_start
    paste_end = frs_end.Offset(1, len(data[0]))

    xl = ws.Application
    
    with screen_lock(xl):
        print("Pasting test data")
        paste_range = cr(paste_start, paste_end)
        paste_range.Clear()
        cr(id_start, id_end).NumberFormat = "@"
        cr(paste_range.Cells(1, 5), paste_range.Cells(len(data), 5)).NumberFormat = "@"
        paste_range.Value2 = data         

        # This has to come before IndentLevel is set,
        # or it gets fucked up for some reason even
        # though it seems to work correctly when performing
        # the operation manually
        print("Applying alignment formatting")
        col = paste_range.Columns(2)
        col.ColumnWidth = 100
        col.VerticalAlignment = xlc.xlTop
        paste_range.Columns(2).WrapText = True
        
        # Vertical alignment should be top for all
        for i in range(1, len(data[0])+1):
            paste_range.Columns(i).VerticalAlignment = xlc.xlTop
            paste_range.Columns(i).HorizontalAlignment = xlc.xlLeft
        
        print("Applying row formatting")
        for d, row in zip(data, paste_range.Rows):
            count = d[0].count(".") * 2
            row.Cells(1,1).IndentLevel = count
            row.Cells(1,2).IndentLevel = count
            if count == 0:
                rint = row.Interior
                rint.Pattern = xlc.xlSolid
                rint.PatternColorIndex = xlc.xlAutomatic
                rint.ThemeColor = xlc.xlThemeColorAccent6
                rint.TintAndShade = 0.6
                
        # Document header row formatting
        rint = paste_range.Rows(1).Interior
        rint.Pattern = xlc.xlSolid
        rint.PatternColorIndex = xlc.xlAutomatic
        rint.ThemeColor = xlc.xlThemeColorDark1
        rint.TintAndShade = -0.249977111117893
        
        print("Applying column autofit")
        # fit after filter to account for width of filter icon
        for i in (1, 3, 4, 5):
            paste_range.Columns(i).AutoFit()

        print("Applying row autofit")
        for r in paste_range.Rows:
            r.EntireRow.AutoFit()

def make_paste_data(root, issues):
    data = [["URS Number", "Text", "References", "Criticality", "Release Version"]]
    for node in root.iter():
        f = node.id
        text = node.text or ""
        id = int(_imkey(f).group(2))
        i = issues[id]
        data.append((f, text, "\n".join(node.refs), i.priority.name, i.sprint_milestone.name))
    return data


In [138]:
from officelib.xllib import *
import re

# unlike the Reference class's item key, the key for the Node class
# is (TYPE+FIRSTNUM, OTHERNUMS). e.g. ('URS123', '4.5') instead of
# ('URS', '123.4.5')
_imkey = re.compile(r"(%s)(\d+)\.?([\d\.]*)" % "|".join(['URS', 'FRS', 'SDS'])).match

def key_match(key):
    m = _imkey(key)
    if m:
        type, first, others = m.groups()
        return type+first, others
    return key, ""

def build_frs_tree(all_items, type):
    root = FRS.Root(key_match)
    for frs, req in all_items.items():
        if req.type != type:
            continue
        node = root.add(frs, int(req.obs))
        node.text = req.text
        node.refs = req.refs
    return root

def main(type='URS', rel_cb=lambda x: True):
    all_frs_items, issues = load_frs_from_issuetracker(rel_cb)
    root = build_frs_tree(all_frs_items, type)
    xl = Excel()
    with screen_lock(xl):
        print("Compiling data for final matrix")
        data = make_paste_data(root, issues)
        
        ws = FRS.get_matrix_sheet(xl)
        paste_data(ws, data)
        print("Done")
    

In [143]:
_ignore = {3194, 3287}
# _cache.clear()
def relevant(i):
    return (i.sprint_milestone == "Legacy" or i.sprint_milestone == '3.0') and i.id not in _ignore and i.status != "Rejected"

trace_path = 'C:\\Users\\Nathan\\Documents\\Dropbox\\FRS'
user_tests = 'FRS Tests 181127.xlsx'
p1 = os.path.join(trace_path, user_tests)
main('URS', relevant)

Compiling data for final matrix
Pasting test data
Applying alignment formatting
Applying row formatting
Applying column autofit
Applying row autofit
Done


In [142]:
xl = Excel()
while True:
    try:
        wb = xl.Workbooks(1)
        wb.Close(False)
    except:
        break
xl.Quit()
del xl

In [86]:
get_issues()[4364].sprint_milestone = '3.0'

In [89]:
get_issues()[3891].status != 'Rejected'

False

In [144]:
root

NameError: name 'root' is not defined